In [ ]:
import numpy 
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import remove_self_loops, add_self_loops, add_remaining_self_loops, degree
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from math import sqrt

In [ ]:
%run preprocessing.ipynb
%run graph.ipynb
data_ns, adj_matrix_ns, conv_layer_ns = create_graph_pyg_ns(master_df, num_neighbors=4, hidden_channels=64)
data_sw, adj_matrix_sw, conv_layer_sw = create_graph_pyg_sw(master_df, hidden_channels=64)

In [ ]:
# Graph Convolution Layer
class GraphConvolution(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(GraphConvolution, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.weight = nn.Parameter(torch.Tensor(input_dim, output_dim))
        self.bias = nn.Parameter(torch.Tensor(output_dim))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)
        nn.init.zeros_(self.bias)

    def forward(self, x, adj):
        support = torch.matmul(x, self.weight)
        output = torch.sparse.mm(adj, support)
        output = output + self.bias
        return F.relu(output)

# GCN Model
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.gc1 = GraphConvolution(input_dim, hidden_dim)
        self.gc2 = GraphConvolution(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x, adj):
        x = self.gc1(x, adj)
        x = self.dropout(x)
        x = self.gc2(x, adj)
        return x

In [ ]:
def rmse(predictions, targets):
    return sqrt(((predictions - targets) ** 2).mean().item())

def train_gcn_model(model, data, adj_matrix, epochs=150, lr=0.01, accumulation_steps=5, clip_value=2,stop_loss=1.5):
    optimizer = Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    mean = data.x.mean(dim=0)
    std = data.x.std(dim=0)
    data.x = (data.x - mean) / std

    losses = []
    rmses =[]

    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        accumulated_loss = 0.0

        for i in range(accumulation_steps):
            output = model(data.x, adj_matrix)
            loss = criterion(output.view(-1), data.y)
            loss.backward()
            accumulated_loss += loss.item() / accumulation_steps

        nn.utils.clip_grad_norm_(model.parameters(), clip_value)
        optimizer.step()

        losses.append(accumulated_loss)

        if accumulated_loss <= stop_loss:
            print(f"Early stopping as accumulated loss went below stop loss")
            print(f"Epoch {epoch + 1},Loss : {accumulated_loss}")
            break

        rmse_val = rmse(output.view(-1), data.y)
        rmses.append(rmse_val)
        print(f'Epoch {epoch + 1},Loss: {accumulated_loss}')
    
    return rmses,losses

In [ ]:
gcn_model = GCN(input_dim=data_ns.x.size(1), hidden_dim=32, output_dim=1)
rmses,losses = train_gcn_model(gcn_model, data_ns, adj_matrix_ns, epochs=200, lr=0.001, accumulation_steps=10, clip_value=1,stop_loss = 1.5)

plt.figure(figsize=(10, 5))
plt.plot(losses, label='Training Loss')
plt.title('Training Loss Per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(rmses, label='RMSE')
plt.title('RMSE Per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
gcn_model = GCN(input_dim=data_sw.x.size(1), hidden_dim=32, output_dim=1)
rmses,losses = train_gcn_model(gcn_model, data_sw, adj_matrix_sw, epochs=200, lr=0.001, accumulation_steps=10, clip_value=1,stop_loss = 1.5)

plt.figure(figsize=(10, 5))
plt.plot(losses, label='Training Loss')
plt.title('Training Loss Per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(rmses, label='RMSE')
plt.title('RMSE Per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()